# QUIZ - 2024-08

In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
from service.models import Service, ServiceCategory, ServiceProvider, Customer, Appointment
from datetime import date, time, datetime

## PART 1 - CRUD Operations
1.1 สร้างบริการ (`Service`) รายละเอียดตามด้านล่าง และเพิ่มเข้าไปในหมวดหมู่ (`ServiceCategory`) ชื่อ "Hair Care"

- Name: "Hair Shower"
- Desciption: "Cold hair shower"
- Price: 100 (Provider 1) และ 200 (Provider 2)
- Service Provider: "Provider 1" และ "Provider 2"

*Hint: ดังนั้นต้องสร้าง 2 แถวในตาราง `Service`*


(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [4]:
# CODE HERE
provider_1 = ServiceProvider.objects.get(name="Provider 1")
provider_2 = ServiceProvider.objects.get(name="Provider 2")

service_1 = Service.objects.create(
    name="Hair Shower",
    description="Cold hair shower",
    price=100,
    service_provider=provider_1
)

service_2 = Service.objects.create(
    name="Hair Shower",
    description="Cold hair shower",
    price=200,
    service_provider=provider_2
)

In [5]:
# Check result
cat = ServiceCategory.objects.first()
cat.services.count()

4

1.2 ทำการแก้ไขราคาของ "Hair Shower" ของ "Provider 1" จาก 100 เป็น 300

(0.25 คะแนน)

In [7]:
# CODE HERE
service = Service.objects.get(name="Hair Shower", service_provider=provider_1)
service.price = 300
service.save()

In [8]:
# Check result
services = Service.objects.filter(name="Hair Shower")
for service in services:
    print(f"Service Name: {service.name}, Provider Name: {service.service_provider.name}, Price: {service.price}")

Service Name: Hair Shower, Provider Name: Provider 2, Price: 200
Service Name: Hair Shower, Provider Name: Provider 1, Price: 300


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-15 โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Massage" ของ "Provider 3" เป็น "Hair Shower" ของ "Provider 1" แทน และทำการเลื่อนวันที่นัดหมายจาก 2024-08-15 เป็น 2024-08-18

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [12]:
# CODE HERE
provider_3 = ServiceProvider.objects.get(name="Provider 3")
massage = Service.objects.get(name="Massage", service_provider=provider_3)

provider_1 = ServiceProvider.objects.get(name="Provider 1")
hair_shower = Service.objects.get(
    name="Hair Shower",
    service_provider=provider_1
)

appointment = Appointment.objects.get(
    appointment_date=date(2024, 8, 15),
    service=massage
)

appointment.service = hair_shower
appointment.appointment_date = date(2024, 8, 18)
appointment.save()

In [13]:
# Check result
res = Appointment.objects.order_by("-appointment_date").first()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-18, Appoint Time: 11:30:00
Service Name: Hair Shower, Provider Name: Provider 1, Customer Name: Customer 1


1.4 ทำตามขั้นตอนดังนี้

1. สร้างผู้ให้บริการ (`ServiceProvider`) ชื่อ "Super Provider" ขึ้นมา (email: super@gmail.com, phone: 089-999-9999) และทำการเพิ่มบริการ (`Service`) ให้กับผู้ให้บริการนี้ (เทพมากสามารถทำได้ทุกบริการ) โดยทุกบริการราคาเท่ากับ 500 บาท

(1 คะแนน)

In [15]:
# CODE HERE
PRICE = 500

service_provider = ServiceProvider.objects.create(
    name="Super Provider",
    email="super@gmail.com",
    phone="089-999-9999"
)

haircut = Service.objects.create(
    service_provider=service_provider,
    name="Haircut",
    description="A simple haircut",
    price=PRICE
)

manicure = Service.objects.create(
    service_provider=service_provider,
    name="Manicure",
    description="Nail trimming and polish",
    price=PRICE
)

massage = Service.objects.create(
    service_provider=service_provider,
    name="Massage",
    description="Full body massage",
    price=PRICE
)

hair_shower = Service.objects.create(
    service_provider=service_provider,
    name="Hair Shower",
    description="Cold hair shower",
    price=PRICE
)

In [16]:
# Check result
for s in Service.objects.filter(service_provider__name="Super Provider"):
    print(f"Service Name: {s.name}, Price: {s.price}, Provider: {s.service_provider.name}")

Service Name: Haircut, Price: 500, Provider: Super Provider
Service Name: Manicure, Price: 500, Provider: Super Provider
Service Name: Massage, Price: 500, Provider: Super Provider
Service Name: Hair Shower, Price: 500, Provider: Super Provider


## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่นัดหมาย (`Appointment.appointment_date`) ในช่วงวันที่ 2024-08-05 ถึง 2024-08-10 ของ "Service Provider 1"

**Expected Output**

จะต้องมี 3 รายการ

(0.5 คะแนน)

In [14]:
# CODE HERE
provider_1 = ServiceProvider.objects.get(name="Provider 1")

dt_start = datetime(2024, 8, 5)
dt_end = datetime(2024, 8, 10)

appointments = Appointment.objects.filter(
    service__service_provider=provider_1,
    appointment_date__range=(dt_start, dt_end)
)

appointments.count()

5

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
```

In [13]:
# CODE HERE
dt_start = datetime(2024, 8, 1)
dt_end = datetime(2024, 8, 2)

appointments = Appointment.objects.filter(
    created_at__range=(dt_start, dt_end)
).order_by("service__service_provider__name")

for appointment in appointments:
    print(f"Name: {appointment.service.name}, Provider: {appointment.service.service_provider.name}")

Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2


2.3 ให้แสดงผลข้อมูลลูกค้า (`Customer`) ดังในตัวอย่าง

```python
[
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [38]:
# CODE HERE
import json

customers = Appointment.objects.values("customer__name").annotate(
    name=F("customer__name"),
    phone=F("customer__phone"),
    appointment_num=Count("customer__name"),
)

cust_list = list(customers.values("name", "phone", "appointment_num"))

print(json.dumps(cust_list, indent=4, sort_keys=False))

[
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]


2.4 ให้แสดงข้อมูลผู้ให้บริการ (`ServiceProvider`) ดังนี้

*หมายเหตุ: Appointment Num หมายถึงจำนวนนัดหมายที่ผู้ให้บริการนั้นมีการได้รับนัด และ Appointed Service หมายถึงบริการที่มีการนัดหมายของผู้ให้บริการนั้นๆ*

(1 คะแนน)

**Expected Output**

```python
Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut
```

In [63]:
# CODE HERE
appointments = Appointment.objects.all().order_by("service__service_provider__name")

# temp
name = ""
appointment_num = 0
appointment_services = []

for appointment in appointments:
    service_provider_name = appointment.service.service_provider.name
    
    if name == "":
        name = service_provider_name
        
    if name != service_provider_name:
        print(f"Name: {name}, Appointment Num: {appointment_num}, Appointed Services: {appointment_services}")
              
        appointment_num = 0
        appointment_services = []
        
        name = service_provider_name
        
    if name == service_provider_name:
        appointment_num += 1
        service_name = appointment.service.name
        appointment_services.append(service_name)
        
print(f"Name: {name}, Appointment Num: {appointment_num}, Appointed Services: {appointment_services}")

Name: Provider 1, Appointment Num: 7, Appointed Services: ['Haircut', 'Massage', 'Massage', 'Manicure', 'Manicure', 'Manicure', 'Massage']
Name: Provider 2, Appointment Num: 5, Appointed Services: ['Manicure', 'Haircut', 'Manicure', 'Massage', 'Manicure']
Name: Provider 3, Appointment Num: 2, Appointed Services: ['Massage', 'Manicure']
Name: Provider 4, Appointment Num: 1, Appointed Services: ['Haircut']


2.5 ให้หาว่าบริการ (`Service`) ไหนของผู้บริการไหนได้รับความนิยมสูงสุด (มีจำนวนนัดหมายมากที่สุด) และ นิยมน้อยที่สุด (มีจำนวนนัดหมายน้อยที่สุด)

(0.5 คะแนน)

**Expected Output**

```python
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Massage, Provider: Provider 4, Appointment Num: 0
Name: Manicure, Provider: Provider 4, Appointment Num: 0
Name: Haircut, Provider: Provider 3, Appointment Num: 0
```

In [89]:
# CODE HERE
appointments = Appointment.objects.values("service__name").annotate(
    sv_name=F("service__name"),
    provider=F("service__service_provider__name"),
    appointment_num=Count("service__name")
).order_by("-appointment_num")

for appointment in appointments:
    print(f"Name: {appointment['sv_name']}, Provider: {appointment['provider']}, Appointment Num: {appointment['appointment_num']}")

Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อบริการ (`Service.name`) เป็นตัวใหญ่ (uppercase) และราคา (`Service.price`) เป็น 2 เท่า และเรียงลำดับตามราคา

(0.5 คะแนน)

**Expected Output**

```python
Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
```

In [70]:
# CODE HERE
from service.models import *
from django.db.models import *
from django.db.models.functions import *

services = Service.objects.all().annotate(
    upper_name=Upper("name"),
    provider=F("service_provider__name"),
    new_price=F("price") * 2
).order_by("provider", "new_price")

for service in services:
    print(f"Name: {service.upper_name}, Provider: {service.provider}, New Price: ${service.new_price}")

Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
